In [9]:
import pandas
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas

df0 = pandas.read_csv('../../data/reddit_wsb.csv')

In [10]:
df = pickle.load(open("../../pickle/text_attr.pkl", "rb"))
df['comms_num'] = df0['comms_num']
print(df.head())

   title_gme  body_gme  title_hold  body_hold  title_buy  body_buy  \
0          0         1           0          0          0         1   
1          1         0           1          0          0         0   
2          0         0           0          0          0         0   
3          0         1           0          0          0         0   
4          0         1           0          0          0         1   

   title_retard  body_retard  title_wife  body_wife  ...  reddit_link  \
0             0            0           0          0  ...            1   
1             0            0           0          0  ...            0   
2             0            0           0          0  ...            1   
3             0            0           0          0  ...            1   
4             0            0           0          0  ...            1   

   yt_link  tweet_link  facebook_link  gif  score  sin_time  cos_time  day  \
0        0           0              0    0      4 -0.983755 -0

# Wycinanie atrybutów po korelacji

In [11]:
reduced_attr_count = 120
corr_matrix = df.corr(method='spearman')

In [12]:
correlations = []
for i,column in enumerate(corr_matrix):
  x = corr_matrix[column][i+1:]
  for row,v in x.items():
    correlations.append([v,column,row])
correlations.sort(key = lambda x: -abs(x[0]))

In [13]:
df_score = df.copy()
score_corr = corr_matrix['score'][corr_matrix['score']<1]
for corr in correlations:
  if len(df_score.columns) <= reduced_attr_count: break
  c,a1,a2 = corr
  if 'score' in [a1,a2]: continue
  if a1 in df_score and a2 in df_score:
    to_be_removed = a1 if score_corr[a1] < score_corr[a2] else a2
    del df_score[to_be_removed]

In [14]:
df_comms_num = df.copy()
comms_num_corr = corr_matrix['comms_num'][corr_matrix['comms_num']<1]
for corr in correlations:
  if len(df_comms_num.columns) <= reduced_attr_count: break
  c,a1,a2 = corr
  if 'comms_num' in [a1,a2]: continue
  if a1 in df_comms_num and a2 in df_comms_num:
    to_be_removed = a1 if comms_num_corr[a1] < comms_num_corr[a2] else a2
    del df_comms_num[to_be_removed]

# Finałowe wycinanie atrybutów po feature importance

In [15]:
final_attr_count = 50
rf = RandomForestRegressor()
rf.fit(df_score[df_score.columns.difference(['score', 'comms_num'])], df_score['score'])

result = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), df_score.head()), 
             reverse=True)
selected = []
for x in result[:final_attr_count]:
    selected.append(x[1])
selected_attributes = df_score[selected + ['score']]
print(selected_attributes)

pickle.dump(selected_attributes, open( "../../pickle/text_attributes_processed_score.pkl", "wb" ))

       title_loss  title_today  title_trade  title_td  title_get  title_🧻  \
0               0            0            0         0          0        0   
1               0            0            0         0          0        0   
2               0            0            0         0          0        0   
3               0            0            0         0          0        0   
4               0            0            0         0          0        0   
...           ...          ...          ...       ...        ...      ...   
38903           0            0            0         0          2        0   
38904           0            0            0         0          0        0   
38905           0            0            0         0          0        0   
38906           0            0            0         0          0        0   
38907           0            0            0         0          0        0   

       title_right  title_know  title_nokia  title_fuck  ...  title_wsb  \


In [16]:
rf = RandomForestRegressor()
rf.fit(df_comms_num[df_comms_num.columns.difference(['score', 'comms_num'])], df_comms_num['comms_num'])

result = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), df_comms_num.head()), 
             reverse=True)
selected = []
for x in result[:final_attr_count]:
    selected.append(x[1])
selected_attributes = df_comms_num[selected + ['comms_num']]
print(selected_attributes)

pickle.dump(selected_attributes, open( "../../pickle/text_attributes_processed_comms_num.pkl", "wb" ))

       body_help  title_day  body_order  title_stop  body_shit  body_gamestop  \
0              0          0           0           0          0              0   
1              0          0           0           0          0              0   
2              0          0           0           0          0              0   
3              0          0           3           0          0              0   
4              0          0           0           0          0              0   
...          ...        ...         ...         ...        ...            ...   
38903          0          0           0           0          0              0   
38904          0          0           0           0          0              0   
38905          0          0           0           0          0              0   
38906          0          0           0           0          0              0   
38907          0          0           0           0          0              0   

       title_hedgies  title